<a href="https://colab.research.google.com/github/neethu2602/DSA_Class_Activities-/blob/main/Dataaquisition_Casestudy_NeethuMathew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [8]:
population=pd.read_csv("/content/population.csv")
population.head()

,country,population
0,United States,1278642419
1,China,792846414
2,India,1001406378
3,Japan,1206263687
4,Germany,428734972


In [7]:
gdp=pd.read_excel("/content/gdp.xlsx")
gdp.head()

,Country,GDP
0,United States,10450436373455
1,China,5086547998939
2,India,14540718118693
3,Japan,12473835857216
4,Germany,2038442396490


In [17]:
internet=pd.read_json("/content/internet_users.json")
internet.head()

,country,internet_users
0,United States,745595490
1,China,685275917
2,India,606850704
3,Japan,1112451555
4,Germany,246827121


In [11]:
literacy_rate=pd.read_xml('/content/literacy_rate.xml')
literacy_rate.head()

,name,literacy_rate
0,United States,72.7
1,China,82.2
2,India,80.3
3,Japan,97.5
4,Germany,92.9


In [14]:
gdp["GDP"] = pd.to_numeric(gdp["GDP"], errors="coerce")
population["population"] = pd.to_numeric(population["population"], errors="coerce")
literacy_rate["literacy_rate"] = pd.to_numeric(literacy_rate["literacy_rate"], errors="coerce")



In [19]:
internet['internet_users']=pd.to_numeric(internet['internet_users'],errors="coerce")


In [24]:
population.isna().sum()

,0
country,0
population,0


In [25]:
gdp.isna().sum()

,0
Country,0
GDP,0


In [26]:
internet.isna().sum()

,0
country,0
internet_users,0


In [27]:
literacy_rate.isna().sum()

,0
name,0
literacy_rate,0


In [29]:
literacy_rate = literacy_rate.rename(columns={"name": "country"})
gdp = gdp.rename(columns={"Country": "country"})

In [36]:
merged = population.merge(internet, on="country", how="outer") \
                       .merge(gdp, on="country", how="outer") \
                       .merge(literacy_rate, on="country", how="outer")

In [37]:
merged.head()

,country,population,internet_users,GDP,literacy_rate
0,Argentina,1432830251,421477197,8767336567320,67.8
1,Australia,439285667,201619113,5444267975247,71.6
2,Brazil,675094950,24027075,8793052816619,81.0
3,Canada,1438267572,894102645,8158351525190,70.8
4,China,792846414,685275917,5086547998939,82.2


In [39]:
merged.isna().sum()

,0
country,0
population,0
internet_users,0
GDP,0
literacy_rate,0


In [47]:
merged["internet_penetration_rate"] = (merged["internet_users"] / merged["population"]) * 100
merged["gdp_per_capita"] = merged["GDP"] / merged["population"]

In [48]:
merged.head()

,country,population,internet_users,GDP,literacy_rate,internet_penetration_rate,gdp_per_capita
0,Argentina,1432830251,421477197,8767336567320,67.8,29.415710,6118.894099
1,Australia,439285667,201619113,5444267975247,71.6,45.897039,12393.456887
2,Brazil,675094950,24027075,8793052816619,81.0,3.559066,13024.912742
3,Canada,1438267572,894102645,8158351525190,70.8,62.165251,5672.346150
4,China,792846414,685275917,5086547998939,82.2,86.432366,6415.552759


In [49]:
duplicates = merged[merged.duplicated()]
duplicates

,country,population,internet_users,GDP,literacy_rate,internet_penetration_rate,gdp_per_capita


In [50]:
print(merged[(merged["population"] < 0) | (merged["internet_users"] < 0) | (merged["GDP"] < 0)])

Empty DataFrame
Columns: [country, population, internet_users, GDP, literacy_rate, internet_penetration_rate, gdp_per_capita]
Index: []


In [51]:
merged.isna().sum()

,0
country,0
population,0
internet_users,0
GDP,0
literacy_rate,0
internet_penetration_rate,0
gdp_per_capita,0


In [55]:
highest_internet = merged.sort_values(by="internet_penetration_rate", ascending=False).head(10)
print(highest_internet[["country", "internet_penetration_rate"]].to_string(index=False))


      country  internet_penetration_rate
        Japan                  92.222917
        Spain                  88.458317
        China                  86.432366
 South Africa                  80.610976
       Canada                  62.165251
        India                  60.599844
United States                  58.311493
      Germany                  57.571026
       Turkey                  52.135863
       France                  49.821208


In [57]:
average_literacy = merged["literacy_rate"].mean()
print(f"Average literacy rate across countries: {average_literacy}%")


Average literacy rate across countries: 78.43%


In [59]:
corr = merged[["GDP", "population", "literacy_rate", "internet_users"]].corr()
print(corr)


                     GDP  population  literacy_rate  internet_users
GDP             1.000000    0.183118      -0.081700        0.212559
population      0.183118    1.000000      -0.229585        0.749573
literacy_rate  -0.081700   -0.229585       1.000000        0.074808
internet_users  0.212559    0.749573       0.074808        1.000000


In [60]:
import sqlite3

In [61]:
conn = sqlite3.connect("data.db")

In [62]:
merged.to_sql("country", conn, if_exists="replace", index=False)

20

In [63]:
cursor = conn.cursor()

cursor.execute("SELECT * FROM country LIMIT 5")
rows = cursor.fetchall()

for row in rows:
    print(row)


('Argentina', 1432830251, 421477197, 8767336567320, 67.8, 29.415710388990107, 6118.894098726005)
('Australia', 439285667, 201619113, 5444267975247, 71.6, 45.89703879412028, 12393.456887467717)
('Brazil', 675094950, 24027075, 8793052816619, 81.0, 3.559066024712524, 13024.912742450524)
('Canada', 1438267572, 894102645, 8158351525190, 70.8, 62.165250917581005, 5672.346150337873)
('China', 792846414, 685275917, 5086547998939, 82.2, 86.4323663321759, 6415.552759174112)


In [71]:
cursor.execute("""SELECT country, internet_penetration_rate
FROM country
ORDER BY internet_penetration_rate DESC
LIMIT 10;""")

In [72]:
high = cursor.fetchall()
for i in high:
    print(i)

('Japan', 92.22291667974251)
('Spain', 88.45831679783196)
('China', 86.4323663321759)
('South Africa', 80.61097645605535)
('Canada', 62.165250917581005)
('India', 60.59984411243684)
('United States', 58.311493418395656)
('Germany', 57.57102571982394)
('Turkey', 52.135862785790486)
('France', 49.82120772511065)


In [75]:
df = pd.read_sql_query("""
SELECT country, internet_penetration_rate
FROM country
ORDER BY internet_penetration_rate DESC
LIMIT 10;
""", conn)

print(df.to_string(index=False))

      country  internet_penetration_rate
        Japan                  92.222917
        Spain                  88.458317
        China                  86.432366
 South Africa                  80.610976
       Canada                  62.165251
        India                  60.599844
United States                  58.311493
      Germany                  57.571026
       Turkey                  52.135863
       France                  49.821208


In [77]:
df2 = pd.read_sql_query("""
SELECT AVG(literacy_rate) AS average_literacy_rate
FROM country;
""", conn)

print(df2.to_string(index=False))

 average_literacy_rate
                 78.43


In [81]:
query = """
SELECT
    (AVG(literacy_rate * internet_penetration_rate) - AVG(literacy_rate) * AVG(internet_penetration_rate)) /
    (SQRT((AVG(literacy_rate * literacy_rate) - AVG(literacy_rate)*AVG(literacy_rate)) *
          (AVG(internet_penetration_rate * internet_penetration_rate) - AVG(internet_penetration_rate)*AVG(internet_penetration_rate))))
    AS correlation
FROM country;
"""

corr_df = pd.read_sql_query(query, conn)
print(corr_df)


   correlation
0      0.23533


In [82]:
df3 = pd.read_sql_query("""SELECT
    (AVG(literacy_rate * internet_penetration_rate) - AVG(literacy_rate) * AVG(internet_penetration_rate)) /
    (SQRT((AVG(literacy_rate * literacy_rate) - AVG(literacy_rate)*AVG(literacy_rate)) *
          (AVG(internet_penetration_rate * internet_penetration_rate) - AVG(internet_penetration_rate)*AVG(internet_penetration_rate))))
    AS correlation
FROM country;
""",conn)
print(df3.to_string(index=False))

 correlation
     0.23533


In [86]:
df4=pd.read_sql_query("""SELECT MIN(gdp_per_capita) AS min_gdp_pc,
       MAX(gdp_per_capita) AS max_gdp_pc,
       AVG(gdp_per_capita) AS avg_gdp_pc,
       COUNT(*) AS country_count
FROM country;
""",conn)
print(df4.to_string(index=False))

 min_gdp_pc    max_gdp_pc   avg_gdp_pc  country_count
3630.082147 132792.830386 19076.768726             20


In [87]:
df5=pd.read_sql_query("""SELECT country, gdp_per_capita
FROM country
WHERE gdp_per_capita > 10000
ORDER BY gdp_per_capita DESC;""",conn)
print(df5.to_string(index=False))

       country  gdp_per_capita
     Indonesia   132792.830386
         Italy    50722.583577
        Russia    26099.651924
        France    17861.109518
United Kingdom    17481.480607
         Spain    16998.007574
         India    14520.297092
        Brazil    13024.912742
     Australia    12393.456887
   South Korea    10609.152180
        Turkey    10437.829234
         Japan    10340.886484


In [89]:
df6=pd.read_sql_query("""SELECT SUM(population) AS total_population
FROM country;""",conn)
print(df6.to_string(index=False))


 total_population
      14864685089


In [91]:
df7=pd.read_sql_query("""SELECT country, literacy_rate, internet_penetration_rate
FROM country
ORDER BY literacy_rate ASC
LIMIT 10;
""",conn)
print(df7.to_string(index=False))


      country  literacy_rate  internet_penetration_rate
 Saudi Arabia           60.5                  11.366429
        Spain           60.6                  88.458317
  South Korea           66.4                  47.165661
    Argentina           67.8                  29.415710
       Canada           70.8                  62.165251
       Turkey           71.4                  52.135863
    Australia           71.6                  45.897039
United States           72.7                  58.311493
    Indonesia           75.4                   4.908162
       Mexico           76.5                  20.746845


In [92]:
df8=pd.read_sql_query("""SELECT country, GDP, population
FROM country
ORDER BY GDP DESC
LIMIT 5;
""",conn)
print(df8.to_string(index=False))


  country            GDP  population
    Spain 15275513099842   898664919
    India 14540718118693  1001406378
    Italy 12907234326850   254467210
    Japan 12473835857216  1206263687
Indonesia 12404144955318    93409749


In [94]:
df9=pd.read_sql_query("""SELECT country, internet_penetration_rate,population
FROM country
WHERE internet_penetration_rate > 70
ORDER BY internet_penetration_rate DESC;
""",conn)
print(df9.to_string(index=False))

     country  internet_penetration_rate  population
       Japan                  92.222917  1206263687
       Spain                  88.458317   898664919
       China                  86.432366   792846414
South Africa                  80.610976   916989541


In [97]:
df10=pd.read_sql_query("""SELECT AVG(gdp_per_capita) AS avg_gdp
FROM country
WHERE internet_penetration_rate > 50;
""",conn)
print(df10.to_string(index=False))

   avg_gdp
8993.62495


In [101]:
df11=pd.read_sql_query("""SELECT COUNT(*) as Number_of_countries,
AVG(internet_penetration_rate) as Average_internet_penetration
from country
where literacy_rate > 90""",conn)
print(df11.to_string(index=False))

 Number_of_countries  Average_internet_penetration
                   3                     66.538383
